In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import warnings
import os
import pandas as pd

warnings.filterwarnings("ignore")

path = r"D:\Computer Vision\yolov5"
os.chdir(path)

from tracker import Tracker


def points(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        print(f"Mouse position: ({x}, {y})")

# cv2.namedWindow("FRAME")  
# cv2.setMouseCallback("FRAME", points)  




# Load model
model = YOLO("yolov8n-pose.pt")


# Load COCO class names
with open(r"coco.txt", "r") as df:
    classes = df.read().split("\n")

    
    
Up = [(63, 99), (291, 63), (315, 100), (95, 131)]
Down = [(643, 614), (813, 547), (795, 525), (631, 588)]


people_Up = {}
people_Down = {}
ss_Up = set()
ss_Down = set()

tracker = Tracker()

overlay_img = cv2.imread(r"graphics-1.png") 
overlay_img = cv2.resize(overlay_img, (300, 100)) 

# Get the dimensions of the overlay image
overlay_height, overlay_width = overlay_img.shape[:2]

path_video = r"people.mp4"
cap = cv2.VideoCapture(path_video)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or cannot read the frame.")
            break

        results = model(frame)
        detections = results[0].boxes 
        if len(detections) == 0:
            print("No detections in this frame.")
            continue

        # Prepare list of bounding boxes for tracking
        lis = []
        for det in detections:
            x1, y1, x2, y2 = map(int, det.xyxy[0])
            conf = float(det.conf[0])
            class_id = int(det.cls[0])
            class_name = classes[class_id]

            # Check if the detected class is a person
            if class_name == 'person':
                lis.append([x1, y1, x2, y2])

        # Update the tracker with the current frame's detections
        bbox_id = tracker.update(lis)

        # Process each detected object
        for bb in bbox_id:
            x3, y3, x4, y4, id = bb

            # Calculate the center point
            cx = (x3 + x4) // 2
            cy = (y3 + y4) // 2
            

            # Check if the point is inside the Up region
            r1 = cv2.pointPolygonTest(np.array(Up, np.int32), (cx, cy), False)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255,255), 3)
            cv2.putText(frame, "Person", (x3, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, str(id), (x3 + 65, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)
            cv2.circle(frame, (cx, cy), 4, (255, 0, 255), -1)

            if r1 >= 0:
                people_Up[id] = (x4, y4)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 255), 3)
                cv2.circle(frame, (cx, cy), 4, (255, 0, 255), -1)
                cv2.putText(frame, "Person", (x3, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                cv2.putText(frame, str(id), (x3 + 65, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)
                ss_Up.add(id)

                
                
            # Check if the point is inside the Down region
            r2 = cv2.pointPolygonTest(np.array(Down, np.int32), (cx, cy), False)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 255), 3)
            cv2.putText(frame, "Person", (x3, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, str(id), (x3 + 65, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)
            cv2.circle(frame, (cx, cy), 4, (255, 0, 255), -1)

            if r2 >= 0:
                people_Down[id] = (x4, y4)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 255), 3)
                cv2.circle(frame, (cx, cy), 4, (255, 0, 255), -1)
                cv2.putText(frame, "Person", (x3, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                cv2.putText(frame, str(id), (x3 + 65, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)
                ss_Down.add(id)

                
        # Overlay image
        pos_x = 915
        pos_y = 375
        if pos_x + overlay_width <= frame.shape[1] and pos_y + overlay_height <= frame.shape[0]:
            frame[pos_y:pos_y+overlay_height, pos_x:pos_x+overlay_width] = overlay_img

        # Draw UP line
        cv2.polylines(frame, [np.array(Up, np.int32)], True, (0, 255, 0), 2)
        cv2.putText(frame, "UP", (951, 353), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
        cv2.putText(frame, str(len(ss_Up)), (1036, 421), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)

        # Draw Down line
        cv2.polylines(frame, [np.array(Down, np.int32)], True, (0, 0, 255), 2)
        cv2.putText(frame, "Down", (1093, 343), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 2)
        cv2.putText(frame, str(len(ss_Down)), (1171, 424), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 2)

        cv2.imshow("FRAME", frame)

        if cv2.waitKey(10) == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
